In [2]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [3]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [4]:
df = pd.read_csv("../data/haberman.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

y[y==1] = 0
y[y==2] = 1

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'haberman'

In [5]:
pd.value_counts(y)

0    225
1     81
Name: count, dtype: int64

In [8]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [130]:
def fit_trees():
    learning_rate=0.1
    max_depth=1
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 1
    verbose = 1
    tolerance=0.001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch= 10
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    # train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    # test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

    # train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    # test_f1 = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)

    train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),gtgp.train_p)
    test_f1 = roc_auc_score(y_test_one_hot.toarray(),gtgp.test_p)


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

7 21 1
10 30 2
12 36 3
13 39 4
15 45 5
16 48 6
17 51 7
17 51 8
17 51 9
17 51 10
17 51 11
17 51 12
17 51 13
17 51 14
17 51 15
17 51 16
17 51 17
17 51 18
17 51 19
17 51 20
17 51 21
17 51 22
17 51 23
17 51 24
17 51 25
17 51 26
17 51 27
17 51 28
17 51 29
17 51 30
17 51 31
17 51 32
17 51 33
17 51 34
17 51 35
17 51 36
17 51 37
17 51 38
17 51 39
17 51 40
17 51 41
17 51 42
17 51 43
17 51 44
17 51 45
17 51 46
17 51 47
17 51 48
17 51 49
17 51 50
17 51 51
17 51 52
17 51 53
17 51 54
17 51 55
17 51 56
17 51 57
17 51 58
17 51 59
17 51 60
17 51 61
17 51 62
17 51 63
17 51 64
17 51 65
17 51 66
17 51 67
17 51 68
17 51 69
17 51 70
17 51 71
17 51 72
17 51 73
17 51 74
17 51 75
17 51 76
17 51 77
17 51 78
17 51 79
17 51 80
17 51 81
17 51 82
17 51 83
17 51 84
17 51 85
17 51 86
17 51 87
17 51 88
17 51 89
17 51 90
17 51 91
17 51 92
17 51 93
17 51 94
17 51 95
17 51 96
17 51 97
17 51 98
17 51 99
17 51 100
retrain  1 :
	train: 0.7336448598130841 77.42450541112657 	test: 0.7391304347826086 33.082590289709465
retrai

# Decision Tree

In [29]:
from sklearn.tree import DecisionTreeClassifier

In [44]:
with open('./benchmark_DC/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        clf = DecisionTreeClassifier(max_depth=2)
        clf.fit(X_train,y_train)
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

        num_trees = 1
        depth = clf.tree_.max_depth
        num_nodes = clf.tree_.node_count

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

        train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [112]:
import xgboost

In [113]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        xgb = xgboost.XGBClassifier(n_estimators=10,max_depth=1)
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))

        # train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

In [42]:
with open('./benchmark_GBDT/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # clf = GradientBoostingClassifier(n_estimators=1000)
        clf = GradientBoostingClassifier(n_estimators=10,max_depth=2)
        clf.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        # train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))


        num_trees = len([ est for ests in clf.estimators_ for est in ests])
        depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
        num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
with open('./benchmark_RF/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # rfc = RandomForestClassifier(n_estimators=100)
        rfc = RandomForestClassifier(n_estimators=10,max_depth=3)
        rfc.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,rfc.predict(X_train))
        test_acc = accuracy_score(y_test,rfc.predict(X_test))
        # train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')
        
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),rfc.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),rfc.predict_proba(X_test))

        num_trees = len(rfc.estimators_)
        depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
        num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)